In [1]:
# Import Libraries
import pandas as pd
import random
from datetime import datetime, timedelta
import numpy as np

In [2]:
# df = pd.read_csv('/Users/cjbertumen/Desktop/SISS-Cashierless-Technology/zEmmanIntegrationTest/integrated.csv')

filename = 'integrated'

df = pd.read_csv(f'./InputDataset/{filename}.csv')
df.rename(columns={'TrackID':'PersonID'}, inplace=True) 
df.rename(columns={'Action':'pickup_or_putdown'}, inplace=True) 
df.rename(columns={'startTime':'Event Time Start'}, inplace=True) 
df.rename(columns={'endTime':'Event Time End'}, inplace=True) 
df.rename(columns={'sensorNumber':'Product'}, inplace=True) 
df.rename(columns={'Frame':'Frame'}, inplace=True) 

filename2 = '10-20_1-HR'
df_ref = pd.read_csv(f'./InputDataset/{filename2}.csv')

In [3]:
df = df.sort_values(by=['PersonID','Event Time Start']).reset_index(drop=True)

In [4]:
# Num of Pickups and Putdowns
pickup_or_putdown = []
# df['pickup_or_putdown'] = np.where((df['Moving Average Start'] >= df['Moving Average End']),"Pickup", "Putback")

sum_Pickup = (df["pickup_or_putdown"]=='Pickup').sum() 
sum_Putdown = (df["pickup_or_putdown"]=='Putback').sum() 

print(f"Pickup: {sum_Pickup}, Putback: {sum_Putdown}")

Pickup: 66, Putback: 41


In [5]:
# Purchase History and Timestamp Allocation Algorithm
PersonShop = []
Purchase = []
PurchaseHistoryPerPerson= []
FinalPurchasePerPerson = []
consolidated_df = []
endTime = []

df['Product'] = df['Product'].dropna().astype(str) # COMMENT DURING FINALIZATION
# df['Product'] = df['Product'].astype(str) #UNCOMMENT DURING FINALIZATION
df['Event Time Start'] = pd.to_datetime(df['Event Time Start'])
df['Event Time End'] = pd.to_datetime(df['Event Time End'])

df['PurchaseHistory'] = None
FinalPurchasePerPersonDetails = []

In [6]:
df['Product'] = df['Product'].replace({'1':'Piattos', '2':'Cream-O', '3':'Whattatops', 
                                      '4':'Loaded', '5':'Bingo', '6':'Lemon Square Cheesecake', 
                                      '7':'Water', '8':'Zesto (Orange)', '9':'Lucky Me (Beef)',
                                      '10':'Mogu Mogu', '11':'Zesto (Apple)', '12':'Lucky Me (Calamansi)'})

In [7]:
df

,Frame,Event Time Start,Event Time End,sensorLocation,Product,weightValue,dateTime,Moving Average,Moving Variance,Event,pickup_or_putdown,PersonID,PurchaseHistory
0,8141,2023-10-20 13:47:10.000325,2023-10-20 13:47:10.486682,B,Zesto (Apple),1600.924075,19650.574427,1601.410598,0.187323,False,Pickup,5,None
1,8255,2023-10-20 13:47:20.000470,2023-10-20 13:47:20.284004,B,Zesto (Orange),1602.938429,19650.574540,1602.847327,0.041215,False,Pickup,5,None
2,8345,2023-10-20 13:47:27.000336,2023-10-20 13:47:27.743755,B,Zesto (Apple),1802.640135,19650.574627,1802.273441,0.127783,False,Putback,5,None
3,8683,2023-10-20 13:48:01.000461,2023-10-20 13:48:01.388284,B,Lucky Me (Beef),606.539449,19650.575016,606.247075,0.118052,False,Pickup,5,None
4,12411,2023-10-20 13:53:39.000346,2023-10-20 13:53:39.485726,A,Bingo,363.091380,19650.578929,363.558436,0.110012,False,Pickup,5,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,37960,2023-10-20 14:31:08.000417,2023-10-20 14:31:08.280557,A,Cream-O,364.962842,19650.604957,364.932984,0.005096,False,Putback,21,None
103,37983,2023-10-20 14:31:10.000371,2023-10-20 14:31:10.271425,A,Bingo,309.724887,19650.604980,309.579715,0.044448,False,Pickup,21,None
104,38099,2023-10-20 14:31:19.000374,2023-10-20 14:31:19.545256,A,Bingo,336.420285,19650.605087,336.552495,0.048749,False,Putback,21,None
105,38214,2023-10-20 14:31:29.000494,2023-10-20 14:31:29.372054,A,Cream-O,331.129741,19650.605201,331.031460,0.021284,False,Pickup,21,None


In [8]:
for i in (df.PersonID.dropna().unique()):
    PersonShop.append(i)

    df_new = df.loc[df['PersonID'].eq(i)]

    for j in range(df[df['PersonID'].eq(i)]['pickup_or_putdown'].count()):
        person = df_new

        if person.iloc[j]['pickup_or_putdown']=='Pickup':
            Purchase.append(person.iloc[j]['Product'])
            endTime.append([person.iloc[j]['Product'],person.iloc[j]['Event Time End']])
            person.iloc[j,12] = str(Purchase)
            person = person
            print(person)

        elif person.iloc[j]['pickup_or_putdown']=='Putback':
            try:
                Purchase.remove(person.iloc[j]['Product'])
                # endTime.remove([person.iloc[j]['Product']])
                person.iloc[j,12] = str(Purchase)
                person = person
            except Exception as e:
                person.iloc[j,12] = str(Purchase)

        if j > (df[df['PersonID'].eq(i)]['pickup_or_putdown'].count()-2):
            consolidated_df.append(person)
            FinalPurchasePerPerson.append(Purchase)
            FinalPurchasePerPersonDetails.append(endTime)
            Purchase = []
            endTime=[]


    Frame           Event Time Start             Event Time End  \
0    8141 2023-10-20 13:47:10.000325 2023-10-20 13:47:10.486682   
1    8255 2023-10-20 13:47:20.000470 2023-10-20 13:47:20.284004   
2    8345 2023-10-20 13:47:27.000336 2023-10-20 13:47:27.743755   
3    8683 2023-10-20 13:48:01.000461 2023-10-20 13:48:01.388284   
4   12411 2023-10-20 13:53:39.000346 2023-10-20 13:53:39.485726   
5   12468 2023-10-20 13:53:44.000558 2023-10-20 13:53:44.514812   
6   12741 2023-10-20 13:54:08.000145 2023-10-20 13:54:08.349565   
7   14667 2023-10-20 13:56:57.000517 2023-10-20 13:56:57.240116   
8   14783 2023-10-20 13:57:06.000236 2023-10-20 13:57:06.646422   
9   14908 2023-10-20 13:57:17.000555 2023-10-20 13:57:17.392072   
10  15143 2023-10-20 13:57:40.000632 2023-10-20 13:57:40.146696   
11  15454 2023-10-20 13:58:07.000515 2023-10-20 13:58:07.322078   
12  22020 2023-10-20 14:07:49.000360 2023-10-20 14:07:49.541931   
13  22218 2023-10-20 14:08:06.000329 2023-10-20 14:08:06.27988

In [9]:
merged_df = pd.DataFrame()

for i in range(len(consolidated_df)):
    merged_df = pd.concat([merged_df, consolidated_df[i]], axis=0)

df_finalPurchase = pd.DataFrame(FinalPurchasePerPersonDetails)


In [10]:
df_finalPurchase

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,"[Zesto (Apple), 2023-10-20 13:47:10.486682]","[Zesto (Orange), 2023-10-20 13:47:20.284004]","[Lucky Me (Beef), 2023-10-20 13:48:01.388284]","[Bingo, 2023-10-20 13:53:39.485726]","[Whattatops, 2023-10-20 13:56:57.240116]","[Bingo, 2023-10-20 13:57:06.646422]","[Piattos, 2023-10-20 13:57:40.146696]","[Lemon Square Cheesecake, 2023-10-20 14:07:49....","[Cream-O, 2023-10-20 14:08:06.279886]","[Mogu Mogu, 2023-10-20 14:08:23.672868]",...,None,None,None,None,None,None,None,None,None,None
1,"[Lemon Square Cheesecake, 2023-10-20 13:53:14....","[Cream-O, 2023-10-20 13:53:22.351366]","[Cream-O, 2023-10-20 13:53:49.258293]","[Cream-O, 2023-10-20 13:57:22.288479]","[Loaded, 2023-10-20 13:57:44.229856]","[Piattos, 2023-10-20 13:58:08.105153]","[Lucky Me (Beef), 2023-10-20 14:03:44.545213]","[Mogu Mogu, 2023-10-20 14:03:55.587403]","[Zesto (Apple), 2023-10-20 14:04:04.509122]","[Zesto (Orange), 2023-10-20 14:04:20.608717]",...,"[Water, 2023-10-20 14:26:15.324759]","[Lucky Me (Beef), 2023-10-20 14:26:36.403165]","[Lucky Me (Calamansi), 2023-10-20 14:26:44.864...","[Water, 2023-10-20 14:27:45.322716]","[Lucky Me (Beef), 2023-10-20 14:30:52.549146]","[Lucky Me (Calamansi), 2023-10-20 14:30:56.547...","[Cream-O, 2023-10-20 14:31:05.207074]","[Bingo, 2023-10-20 14:31:10.271425]","[Cream-O, 2023-10-20 14:31:29.372054]","[Mogu Mogu, 2023-10-20 14:31:38.455375]"


In [11]:
merged_df

,Frame,Event Time Start,Event Time End,sensorLocation,Product,weightValue,dateTime,Moving Average,Moving Variance,Event,pickup_or_putdown,PersonID,PurchaseHistory
0,8141,2023-10-20 13:47:10.000325,2023-10-20 13:47:10.486682,B,Zesto (Apple),1600.924075,19650.574427,1601.410598,0.187323,False,Pickup,5,['Zesto (Apple)']
1,8255,2023-10-20 13:47:20.000470,2023-10-20 13:47:20.284004,B,Zesto (Orange),1602.938429,19650.574540,1602.847327,0.041215,False,Pickup,5,"['Zesto (Apple)', 'Zesto (Orange)']"
2,8345,2023-10-20 13:47:27.000336,2023-10-20 13:47:27.743755,B,Zesto (Apple),1802.640135,19650.574627,1802.273441,0.127783,False,Putback,5,['Zesto (Orange)']
3,8683,2023-10-20 13:48:01.000461,2023-10-20 13:48:01.388284,B,Lucky Me (Beef),606.539449,19650.575016,606.247075,0.118052,False,Pickup,5,"['Zesto (Orange)', 'Lucky Me (Beef)']"
4,12411,2023-10-20 13:53:39.000346,2023-10-20 13:53:39.485726,A,Bingo,363.091380,19650.578929,363.558436,0.110012,False,Pickup,5,"['Zesto (Orange)', 'Lucky Me (Beef)', 'Bingo']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,37960,2023-10-20 14:31:08.000417,2023-10-20 14:31:08.280557,A,Cream-O,364.962842,19650.604957,364.932984,0.005096,False,Putback,21,"['Cream-O', 'Mogu Mogu', 'Lucky Me (Calamansi)..."
103,37983,2023-10-20 14:31:10.000371,2023-10-20 14:31:10.271425,A,Bingo,309.724887,19650.604980,309.579715,0.044448,False,Pickup,21,"['Cream-O', 'Mogu Mogu', 'Lucky Me (Calamansi)..."
104,38099,2023-10-20 14:31:19.000374,2023-10-20 14:31:19.545256,A,Bingo,336.420285,19650.605087,336.552495,0.048749,False,Putback,21,"['Cream-O', 'Mogu Mogu', 'Lucky Me (Calamansi)..."
105,38214,2023-10-20 14:31:29.000494,2023-10-20 14:31:29.372054,A,Cream-O,331.129741,19650.605201,331.031460,0.021284,False,Pickup,21,"['Cream-O', 'Mogu Mogu', 'Lucky Me (Calamansi)..."


In [12]:
# Manipulation to find the timestamp of each purchased item
manipulated=[]
manipulatedPerPerson = []
flag = 0
for idx, _ in enumerate(FinalPurchasePerPersonDetails):
    for _ , name in enumerate(FinalPurchasePerPerson[idx]):
        for j in reversed(range(len(FinalPurchasePerPersonDetails[idx]))):
            if FinalPurchasePerPersonDetails[idx][j][0] == name:
                manipulated.append(FinalPurchasePerPersonDetails[idx][j])
                del FinalPurchasePerPersonDetails[idx][j]
                break
    manipulatedPerPerson.append(manipulated)
    manipulated = [] 

In [13]:
df_ref['startTime'] = pd.to_datetime(df_ref['startTime'])

In [15]:
## Average Time Proper
PersonIDList = []
df_tmp = []
df2 = pd.DataFrame()

for idx, _ in enumerate(manipulatedPerPerson):
    df_tmp = pd.DataFrame(manipulatedPerPerson[idx])
    df2 = pd.concat((df2, df_tmp), axis=0)
# df_productTimeStampHistory = pd.DataFrame(manipulatedPerPerson,columns=[[Product','endTime'])
df2.columns=['Product','endTime']
df2['Event Time Start'] = df_ref.loc[:,'startTime'].iloc[0]
df2['TimeSpentOnShelf'] = df2['endTime'] - df2['Event Time Start']
# df2['Product'] = df2['Product'].astype(int)
df2_grouped = df2.groupby(by='Product')

In [16]:
merged_df.to_csv(f'./OutputResults/{filename}_Complete_Results_and_History.csv')

df2_grouped_to_output = pd.DataFrame(df2_grouped.mean().sort_values(by="Product",ascending=True))
df2_grouped_to_output.to_csv(f'./OutputResults/{filename}_avg_time_per_product.csv')